# Superdense Coding Kata

**Superdense Coding** quantum kata is a series of exercises designed to get you familiar with programming in Q#.

It covers the superdense coding protocol which allows us to transmit two bits of classical information by sending just one qubit using previously shared quantum entanglement.

 - A good description can be found in [the Wikipedia article](https://en.wikipedia.org/wiki/Superdense_coding).
 - A great interactive demonstration can be found [on the Wolfram Demonstrations Project](http://demonstrations.wolfram.com/SuperdenseCoding/).
 - Superdense coding protocol is described in Nielsen & Chuang, section 2.3 (pp. 97-98).

Each task is wrapped in one operation preceded by the description of the task. Your goal is to fill in the blank (marked with `// ...` comment) with some Q# code that solves the task. To verify your answer, run the cell using Ctrl/⌘+Enter.

Each task defines an operation that can be used in subsequent tasks to simplify implementations and build on existing code.  We split the superdense coding protocol into several steps, following the description in the [Wikipedia article](https://en.wikipedia.org/wiki/Superdense_coding):

* Preparation (creating the entangled pair of qubits that are sent to Alice and Bob).
* Encoding the message (Alice's task): Encoding the classical bits of the message into the state of Alice's qubit which then is sent to Bob.
* Decoding the message (Bob's task): Using Bob's original qubit and the qubit he received from Alice to decode the classical message sent.
* Finally, we compose those steps into the complete superdense coding protocol.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [1]:
%package Microsoft.Quantum.Katas::0.10.1911.1607

Microsoft.Quantum.Standard::0.10.1911.1607, Microsoft.Quantum.Katas::0.10.1911.1607

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


### Task 1. Entangled pair

**Input:** Two qubits, each in the $|0\rangle$ state.

**Goal:**  Prepare a Bell state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)$ on these qubits.

In [2]:
%kata T1_CreateEntangledPair_Test

operation CreateEntangledPair (q1 : Qubit, q2 : Qubit) : Unit is Adj {
    H(q1);
    CNOT(q1, q2);
}

Success!

### Task 2. Send the message (Alice's task) 

Encode the message (two classical bits) in the state of Alice's qubit.

**Inputs**: 
1. Alice's part of the entangled pair of qubits qAlice.
2. Two classical bits, stored as ProtocolMessage.

**Goal**:  Transform the input qubit to encode the two classical bits.

>`ProtocolMessage` is a custom type that represents the message to be transmitted. It includes two items of type `Bool` called `Bit1` and `Bit2`.

<br/>
<details>
  <summary>Need a hint? Click here</summary>
Manipulate Alice's half of the entangled pair to change the joint state of the two qubits to one of the following four states based on the value of message:   

* [0; 0]: $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)$
* [0; 1]: $|\Psi^{+}\rangle = \frac{1}{\sqrt{2}} (|01\rangle + |10\rangle)$
* [1; 0]: $|\Phi^{-}\rangle = \frac{1}{\sqrt{2}} (|00\rangle - |11\rangle)$
* [1; 1]: $|\Psi^{-}\rangle = \frac{1}{\sqrt{2}} (|01\rangle - |10\rangle)$

</details>

📝 $00, 01, 10, 11$ を、それぞれ $|\Phi^{+}\rangle, |\Psi^{+}\rangle, |\Phi^{-}\rangle, |\Psi^{-}\rangle$ に対応づける。

In [3]:
%kata T2_EncodeMessageInQubit_Test 

open Quantum.Kata.SuperdenseCoding;

operation EncodeMessageInQubit (qAlice : Qubit, message : ProtocolMessage) : Unit {
    if (message::Bit1) { // accesses the item 'Bit1' of 'message'
        Z(qAlice);
    }
    if (message::Bit2) {
        X(qAlice);
    }
}

Success!

### Task 3. Decode the message and reset the qubits (Bob's task)

Decode the message using the qubit received from Alice and reset both qubits to a $|00\rangle$ state.

**Inputs:**

1. Qubit received from Alice qAlice.
2. Bob's part of the entangled pair qBob.

**Goal** :  Retrieve two bits of classic data from the qubits and return them as `ProtocolMessage`. The state of the qubits in the end of the operation should be $|00\rangle$.

> You can create an instance of `ProtocolMessage` as `ProtocolMessage(bit1value, bit2value)`.

📝 `CNOT` と `H` を使うと、

$$
\begin{align}
|\Phi^{+}\rangle &= \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)
    \longrightarrow \frac{1}{\sqrt{2}} (|00\rangle + |10\rangle)
    \longrightarrow \frac{1}{2} (|00\rangle + |10\rangle + |00\rangle - |10\rangle)
    = |00\rangle \\
|\Psi^{+}\rangle &= \frac{1}{\sqrt{2}} (|01\rangle + |10\rangle)
    \longrightarrow \frac{1}{\sqrt{2}} (|01\rangle + |11\rangle)
    \longrightarrow \frac{1}{2} (|01\rangle + |11\rangle + |01\rangle - |11\rangle)
    = |01\rangle \\
|\Phi^{-}\rangle &= \frac{1}{\sqrt{2}} (|00\rangle - |11\rangle)
    \longrightarrow \frac{1}{\sqrt{2}} (|00\rangle - |10\rangle)
    \longrightarrow \frac{1}{2} (|00\rangle + |10\rangle - |00\rangle + |10\rangle)
    = |10\rangle \\
|\Psi^{-}\rangle &= \frac{1}{\sqrt{2}} (|01\rangle - |10\rangle)
    \longrightarrow \frac{1}{\sqrt{2}} (|01\rangle - |11\rangle)
    \longrightarrow \frac{1}{2} (|01\rangle + |11\rangle - |01\rangle + |11\rangle)
    = |11\rangle \\
\end{align}
$$

In [8]:
%kata T3_DecodeMessageFromQubits_Test 

open Quantum.Kata.SuperdenseCoding;
open Microsoft.Quantum.Convert;

operation DecodeMessageFromQubits (qAlice : Qubit, qBob : Qubit) : ProtocolMessage {
    CNOT(qAlice, qBob);
    H(qAlice);
    
    let bit1 = ResultAsBool(M(qAlice));
    let bit2 = ResultAsBool(M(qBob));
    
    // リセット
    if (bit1) {
        X(qAlice);
    }
    if (bit2) {
        X(qBob);
    }

    return ProtocolMessage(bit1, bit2);
}

Success!

📝 `ReferenceImplementation.qs` の解答例を見てみると、最初の `CNOT`・`H` は Task 1 の逆操作なので、

```
Adjoint CreateEntangledPair(qAlice, qBob);
```

が使えるらしい。

また [`MResetZ`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.measurement.mresetz?view=qsharp-preview) を使うと、「測定してリセット」が一度に行えるとのこと。

### Task 4. Superdense coding protocol end-to-end:

Put together the steps performed in tasks 1-3 to implement the full superdense coding protocol.

**Input:** Two classical bits to be transmitted.

**Goal:**  Prepare an EPR Pair, encode the two classical bits in the state of the pair by applying quantum gates to one member of the pair, and decode the two classical bits from the state of the pair. Return the result of decoding.

📝 これまでの手順をまとめるだけ。

In [9]:
%kata T4_SuperdenseCodingProtocol_Test

open Quantum.Kata.SuperdenseCoding;

operation SuperdenseCodingProtocol (message : ProtocolMessage) : ProtocolMessage {
    using ((qAlice, qBob) = (Qubit(), Qubit())) {
        CreateEntangledPair(qAlice, qBob);
        EncodeMessageInQubit(qAlice, message);
        return DecodeMessageFromQubits(qAlice, qBob);
    }
}

Success!